# Post-processing

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from ast import literal_eval
import numpy as np
from pathlib import Path
from src.utils import kfold_post_process, create_heatmap, prepare_df, rename, n_random_post_process
import seaborn as sns
# 0.0001 5e-6
# 2.5e-6 2.5e-7
# 0.0005 2.5e-6
# 5e-6 2.5e-7

In [2]:
ft_dir = Path("results/mixed/")

base_df = pd.read_csv("results/mlaad_analysis/baseline_evaluation_results.csv")
# fine_df = pd.concat([
#     pd.read_csv(ft_dir / "1_evaluation_results.csv"),
#     pd.read_csv(ft_dir / "2_evaluation_results.csv")
# ]
    
# )

reverse_dir = Path("results/mlaad_analysis_reversed")
ft_dir = Path("results/mlaad_analysis/")
fine_df = pd.concat(
    
    [
        pd.read_csv(ft_dir / "1_evaluation_results.csv"), 
        pd.read_csv(ft_dir / "2_evaluation_results.csv"), 
        pd.read_csv(reverse_dir / "1_final_evaluation_results.csv"), 
        pd.read_csv(reverse_dir / "2_final_evaluation_results.csv")
        ]
    )

# fine_df = pd.read_csv("results/mlaad_trained/evaluation_results_final.csv")
base_df = prepare_df(base_df)
fine_df = prepare_df(fine_df)


In [ ]:
gruped_df = fine_df.groupby(['model',  'fine-tuned languages', 'evaluated languages']).apply(lambda x:x)


### To aggragete the rows (if we use both cases -> D_ft to ft and D_eval to eval and inversely)

In [4]:
fine_df = fine_df.groupby(['model',  'fine-tuned languages', 'evaluated languages'], as_index=False).agg({
    'y': lambda x: sum(x, []), 
    'y_pred': lambda x: sum(x, []), 
    'y_pred_label': lambda x: sum(x, [])
})


In [ ]:

baseline_df = n_random_post_process(base_df, 5, 0.9)
ft_df = n_random_post_process(fine_df, 5, 0.9)

In [7]:
ft_df['fine-tuned languages'] = ft_df['fine-tuned languages'].apply(literal_eval).apply(lambda x: "+".join(x))
baseline_df['fine-tuned languages'] = '-'

In [8]:
ft_df = ft_df.loc[ft_df['fine-tuned languages'].apply(lambda x: len(x)) > 0]

In [9]:
ft_df['model_id'] = ft_df['model'] + " ft with " + ft_df['fine-tuned languages']
baseline_df['model_id'] = "ASV trained " + baseline_df['model']

In [ ]:
baseline_df = baseline_df.groupby(['model', 'model_id', 'fine-tuned languages', 'evaluated languages'])[baseline_df.select_dtypes(include=[np.number]).columns].mean()
ft_df = ft_df.groupby(['model', 'model_id', 'fine-tuned languages', 'evaluated languages'])[ft_df.select_dtypes(include=[np.number]).columns].mean()
ft_df.head()

In [ ]:
baseline_df.head()

In [12]:
df = pd.concat([baseline_df, ft_df])


In [13]:
df = df.apply(lambda x: x).reset_index()

In [14]:
df['eer'] = df['eer'].apply(lambda x: round(100*x, 2))
df['accuracy'] = df['accuracy'].apply(lambda x: round(100*x, 2))

In [ ]:
df.pivot(index=['model_id'], columns='evaluated languages', values=['eer'])

In [ ]:
def create_heatmap(df, model_name, metric='eer', metric_label="EER", maximize=False, output_dir="results/plots", title_prefix=""):
    dataframe = df.copy()
    heatmap_data = dataframe.loc[dataframe['model'] == model_name].pivot(index='fine-tuned languages', columns='evaluated languages', values=metric)
    
    plt.figure(figsize=(10, 10))
    
    ax = sns.heatmap(
        heatmap_data,
        annot=True,
        cmap="Blues",
        cbar_kws={'label': metric_label},
        annot_kws={"size": 20},
        fmt='.2f'
    )
    plt.xlabel('Evaluated with', fontsize=20)
    plt.ylabel('Fine-tuned with', fontsize=20)
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)

    if '-' in heatmap_data.index:
        idx = heatmap_data.index.get_loc('-')
        n_rows = len(heatmap_data.index)
        n_cols = len(heatmap_data.columns)
        ax.add_patch(plt.Rectangle((0, idx), n_cols, 1, fill=False, edgecolor='black', lw=4))

        ax.vlines(x=[0, 0], ymin=0, ymax=0, colors='black', linewidth=4)

    for col in heatmap_data.columns:
        if maximize:
            best_value = heatmap_data[col].max()
        else:
            best_value = heatmap_data[col].min()

        # Find all indices of the best value in the column
        best_indices = heatmap_data.index[heatmap_data[col] == best_value]

        for idx in best_indices:
            # Find row and column index to add the rectangle patch
            row_idx = heatmap_data.index.get_loc(idx)
            col_idx = heatmap_data.columns.get_loc(col)

            # Draw a red rectangle around the best cells in the column
            ax.add_patch(plt.Rectangle((col_idx, row_idx), 1, 1, fill=False, edgecolor='red', lw=3))

    # model_name = model_name.lower()
    
    model_name = rename(model_name)
    cbar = ax.collections[0].colorbar
    cbar.ax.tick_params(labelsize=20)
    cbar.set_label(metric_label, fontsize=20)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])

    plt.title(f'{metric_label} for {model_name}', fontsize=18)
    print(f"{output_dir}/{title_prefix}/{title_prefix}{model_name.replace('+', '-')}.png")
    plt.savefig(f"{output_dir}/{title_prefix}{model_name.replace('+', '-')}.png")
    plt.show()
    
for name in df.apply(lambda x: x)['model'].unique():
    # create_heatmap(df.loc[df['fine-tuned languages']!= '-'], name, title_prefix="trained")
    create_heatmap(df, name, title_prefix="")

# EER for model 

# Top 10 models for evaluated language

In [ ]:
from pandas import DataFrame

k_base = n_random_post_process(base_df, 5, 0.9)
k_ft = n_random_post_process(fine_df, 5, 0.9)
cols = ['model', 'fine-tuned languages', 'evaluated languages', 'eer']
k_base['model'] = k_base['model'] + ' trained on ASV'
k_ft = k_ft.loc[k_ft['fine-tuned languages'].apply(lambda x: len(x)) > 4]

dataframe = pd.concat([k_base[cols], k_ft[cols]])
dataframe['fine-tuned languages'] = dataframe['fine-tuned languages'].apply(literal_eval).apply(lambda x: "+".join(x))

dataframe['model'] = dataframe.apply(lambda x: x['model'] + ' fine-tuned with ' + x['fine-tuned languages'] if "ASV" not in x['model'] else x['model'], axis=1)
dataframe['model'] = dataframe['model'].apply(rename)


In [20]:
dataframe = dataframe.groupby(['model', 'fine-tuned languages', 'evaluated languages']).agg(lambda x: f"{100*x.mean():.2f} ± {100*x.std():.2f}")

dataframe: DataFrame  = dataframe.pivot_table(values='eer', index='model', columns=['evaluated languages'], aggfunc=''.join, fill_value='-')

In [ ]:
dataframe

In [ ]:
lan = 'en'
dataframe[lan].apply(lambda x: float(x.split(' ')[0]) if x != '-' else np.inf).sort_values().head(10)

In [ ]:
lan = 'de'
dataframe[lan].apply(lambda x: float(x.split(' ')[0]) if x != '-' else np.inf).sort_values().head(10)

In [ ]:
lan = 'fr'
dataframe[lan].apply(lambda x: float(x.split(' ')[0]) if x != '-' else np.inf).sort_values().head(10)

In [ ]:
lan = 'it'
dataframe[lan].apply(lambda x: float(x.split(' ')[0]) if x != '-' else np.inf).sort_values().head(10)

In [ ]:
lan = 'es'
dataframe[lan].apply(lambda x: float(x.split(' ')[0]) if x != '-' else np.inf).sort_values().head(10)

In [ ]:
lan = 'pl'
dataframe[lan].apply(lambda x: float(x.split(' ')[0]) if x != '-' else np.inf).sort_values().head(10)

In [ ]:
lan = 'uk'
dataframe[lan].apply(lambda x: float(x.split(' ')[0]) if x != '-' else np.inf).sort_values().head(10)

In [ ]:
lan = 'ru'
dataframe[lan].apply(lambda x: float(x.split(' ')[0]) if x != '-' else np.inf).sort_values().head(10)

In [ ]:
langs = ['en', 'de']
langs = ['fr', 'es', 'it']
langs = ['pl', 'ru', 'uk']

dataframe[langs]

In [ ]:
langs = ['en', 'de']
# langs = ['fr', 'es', 'it']
# langs = ['pl', 'ru', 'uk']
caption = "Baseline"
latex_code = dataframe.loc[dataframe.index.str.contains('ASV')][langs].to_latex(index=True, column_format='l|ccc', float_format="%.2f", caption=caption, label=f"tab:{'_'.join(langs)}")
latex_code = latex_code.replace("trained on ASV", "&")
print(latex_code)

In [ ]:
langs = ['en', 'de']
# langs = ['fr', 'es', 'it']
# langs = ['pl', 'ru', 'uk']
caption = "The mean EER scores with standard deviation for $K$-fold tests for $K=5$ for the Germanic languages tested on ASV-trained models."
latex_code = dataframe.loc[dataframe.index.str.contains('ASV')][langs].to_latex(index=True, column_format='l|cc', float_format="%.2f", caption=caption, label=f"tab:{'_'.join(langs)}")
print(latex_code)


In [ ]:
langs = ['en', 'de']
caption = "The mean EER scores with standard deviation for $K$-fold tests for $K=5$ for the Germanic languages tested on ASV-trained models fine-tuned with samples generated using $D_{ft}$ subset for the specific language."

model, i = 'LFCC', 2

latex_code =  dataframe.loc[(~dataframe.index.str.contains('ASV')) & (dataframe.index.str.contains(model))][langs].to_latex(index=True, column_format='l|ccc', float_format="%.2f", caption=caption, label=f"tab:{'_'.join(langs)}")
print(latex_code.replace(f"{model} fine-tuned with ", f"{''.join(langs)}&"))

In [ ]:
langs = ['fr', 'es', 'it']
caption = "The mean EER scores with standard deviation for $K$-fold tests for $K=5$ for the Germanic languages tested on ASV-trained models fine-tuned with samples generated using $D_{ft}$ subset for the specific language."

latex_code =  dataframe.loc[(~dataframe.index.str.contains('ASV')) & (dataframe.index.str.contains(model))][langs].to_latex(index=True, column_format='l|ccc', float_format="%.2f", caption=caption, label=f"tab:{'_'.join(langs)}")
print(latex_code.replace(f"{model} fine-tuned with ", f"{''.join(langs)}&"))